In [21]:
import json
import psycopg2
from datetime import datetime

In [6]:
## Set up connection
connection = psycopg2.connect(host='localhost',
                             user='postgres',
                             port=5432,
                             database="CSCI 3907 Final Project",
                             password='postgres')

connection.set_session(autocommit=True)

cur = connection.cursor()

In [15]:
## Read from file
with open('data/kindle_reviews.json') as f:
    data = [json.loads(line) for line in f]

## Insertion from file
print(data[0])
query = "INSERT INTO users VALUES (%s, %s)"
cur.execute(query, (data[0]['reviewerID'],data[0]['reviewerName']))
connection.commit()

cur.execute("SELECT * FROM users")
result = cur.fetchall()
print(result)

cur.execute("DELETE FROM users WHERE uid=%s",(data[0]['reviewerID'],))

cur.execute("SELECT * FROM users")
result = cur.fetchall()
print(result)

{'reviewerID': 'A1F6404F1VG29J', 'asin': 'B000F83SZQ', 'reviewerName': 'Avidreader', 'helpful': [0, 0], 'reviewText': "I enjoy vintage books and movies so I enjoyed reading this book.  The plot was unusual.  Don't think killing someone in self-defense but leaving the scene and the body without notifying the police or hitting someone in the jaw to knock them out would wash today.Still it was a good read for me.", 'overall': 5.0, 'summary': 'Nice vintage story', 'unixReviewTime': 1399248000, 'reviewTime': '05 5, 2014'}
[('A1F6404F1VG29J', 'Avidreader')]
[]


In [51]:
## Queries
## These check for duplicate key entries using 'on conflict'
users_query = "INSERT INTO users VALUES (%s, %s) ON CONFLICT DO NOTHING"
prods_query = "INSERT INTO products VALUES (%s) ON CONFLICT DO NOTHING"
reviews_query = "INSERT INTO reviews VALUES (%s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT DO NOTHING"

## Insert all data
for i in range(len(data)):
    uid = data[i]['reviewerID']
    pid = data[i]['asin']
    
    ## Some records don't have names replace with null
    try:
        name = data[i]['reviewerName']
    except KeyError:
        name = None
        
    posHelpful = data[i]['helpful'][0]
    negHelpful = data[i]['helpful'][1]
    
    ## Some records have null characters replace with blank
    text = data[i]['reviewText']
    text = str.replace(text, "\u0000", "", -1)
    
    rating = data[i]['overall']
    summary = data[i]['summary']
    time_str = data[i]['reviewTime']
    time = datetime.strptime(time_str, '%m %d, %Y').date()
        
    cur.execute(users_query, (uid, name))
    connection.commit()
    cur.execute(prods_query, (pid,))
    connection.commit()
    cur.execute(reviews_query, (uid, pid, text, summary, rating, posHelpful, negHelpful, time))
    connection.commit()